s## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

- Гринда Юліана
- Павлосюк Роман

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [ ]:
class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __repr__(self) -> str:
        return f'{self.char}: {self.freq}'

class Huffman:
    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        letters = {letter: '' for letter in text}
        nodes = sorted([Node(letter, text.count(letter) /
                             len(text)) for letter in letters], key = lambda x: x.freq)
        tree = self.build_tree(nodes)
        alphabet = self.alphabet_code(tree, letters)
        result = ''
        for char in text:
            result += alphabet[char]
        return (result, alphabet)
    
    def build_tree(self, nodes):
        while len(nodes) > 1:
            left_node = nodes.pop(0)
            right_node = nodes.pop(0)
            parent_node = Node(None, left_node.freq + right_node.freq)
            parent_node.left = left_node
            parent_node.right = right_node
            nodes.append(parent_node)
            nodes = sorted(nodes, key = lambda x: x.freq)

        return nodes[0]

    def alphabet_code(self, node, letters, code = ''):
        if not node.char is None:
            letters[node.char] = code
        else:
            for i in range(2):
                if i == 0:
                    code_ = code + '0'
                    self.alphabet_code(node.left, letters, code_)
                else:
                    code_ = code + '1'
                    self.alphabet_code(node.right, letters, code_)
        return letters

    def decode(self, code: str, coding_dict: dict[str, str]):
        coding_dict = {value: key for key, value in coding_dict.items()}
        result = ''
        match_ = ''
        for char in code:
            if match_ + char in coding_dict:
                result += coding_dict[match_ + char]
                match_ = ''
            else:
                match_ += char
        return result


# Алгоритм LZW

In [ ]:
class LZW:
    def encode(self, text:str) -> tuple[str, list]:
        '''LZW encoding'''
        output = []
        w = ''
        start_dictionary = {}
        counter = 0
        for letter in text:
            if letter not in start_dictionary:
                start_dictionary[letter] = counter
                counter += 1
        new_dictionary = start_dictionary.copy()
        for letter in text:
            if w + letter in new_dictionary:
                w += letter
            else:
                new_dictionary.update({w + letter: max(new_dictionary.values())+1})
                output.append(new_dictionary[w])
                w = letter
        output.append(new_dictionary[w])
        return output, list(start_dictionary.keys())

    def decode(self, code: list, coding_dict: list) -> str:
        '''LZW decoding'''
        coding_dict = {i:el for i,el in enumerate(coding_dict)}
        string = coding_dict[code[0]]
        output = ''
        output += string
        counter = max(coding_dict.keys()) + 1
        for i in range(len(code)-1):
            new = code[i + 1]
            if new not in coding_dict:
                entry = string + string[0]
            else:
                entry = coding_dict[new]
            output += entry
            coding_dict[counter] = string + entry[0]
            counter += 1
            string = entry
        return output


# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [ ]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size

    def encode(self, text: str) -> str:
        encoded_data = []
        current_index = 0

        while current_index < len(text):
            best_offset = -1
            best_length = -1
            best_match = ''

            for length in range(1, min(len(text) - current_index, self.buffer_size)):
                sub_data = text[current_index:current_index + length]
                offset = text.rfind(sub_data, max(0, current_index - self.buffer_size), current_index)

                if offset != -1 and length > best_length:
                    best_offset = current_index - offset
                    best_length = length
                    best_match = sub_data

            if best_match:
                encoded_data.append((best_offset, best_length, text[current_index + best_length]))
                current_index += best_length + 1
            else:
                encoded_data.append((0, 0, text[current_index]))
                current_index += 1

        return encoded_data

    def decode(self, code: str) -> str:
        decoded_data = []

        for item in code:
            offset, length, code_word = item

            if not length:
                decoded_data.append(code_word)
            else:
                start = len(decoded_data) - offset
                substring = decoded_data[start:start + length]
                decoded_data.extend(substring)
                decoded_data.append(code_word)

        return ''.join(decoded_data)

# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size: int):
        pass

    def encode(self, text: str) -> str:
        pass

    def decode(self, code: str) -> str:
        pass